## Simple Streamlit App for Cortex LLM

Simple app showing off how to automatically save prompts / responses for evaluation of Cortex LLM.

I created a table `cortex_eval_table` with columns for PROMPT, RESPONSE, MODEL, and DATE

In [ ]:
# Import python packages
import streamlit as st
from datetime import datetime
from streamlit import cache_resource, cache_data
from snowflake.snowpark.session import Session
from snowflake.snowpark.context import get_active_session
import re
import json

session = get_active_session()

@cache_data(show_spinner='CALLING LLM')
def complete(model, prompt):
    global session
    prompt = re.sub("'", "''", prompt)
    sql_code = f"SELECT SNOWFLAKE.CORTEX.COMPLETE('{model}', '{prompt}') as answer"
    df = session.sql(sql_code).to_pandas()
    return df['ANSWER'][0], sql_code

st.header("Evaluation App with Cortex")
st.subheader("Prompt the LLM, all prompts and responses are save")
completion_input = st.text_area("Prompt",height=400)
completion_model = st.selectbox("Choose Model", options = ['llama2-70b-chat','mistral-7b'])

if st.button("Complete"):
    if completion_input is not None and completion_model is not None:
        result, sql_code = complete(completion_model,completion_input)
        st.success(result)
        df = session.create_dataframe([[completion_input, result, completion_model, datetime.now().date()]], schema=["PROMPT", "RESPONSE", "MODEL", "DATE"])
        df.write.save_as_table("cortex_eval_table", mode="append")
        with st.expander("See SQL"):
            st.code(sql_code,language='SQL') 